In [1]:
#####################################################
# 3.6　サンプルの重みを使って画像分類をしてみる
#####################################################

import sys, os
sys.path.append(os.pardir)

from dataset.mnist import load_mnist
from common.functions import sigmoid, softmax #sigmoid/softmaxはテキストのやつを借りちゃいます。

import pickle
import numpy as np

################################
#基本関数群の定義
################################

#テスト用のデータを読み込んで返す
def get_data():
    (x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, flatten=True, one_hot_label=False)
    return x_test, t_test

#neural networkを初期化する
def init_network():
    with open("sample_weight.pkl", "rb") as f:
        network = pickle.load(f)
        
    return network

#neural networkを使って推論する関数。合計４層のネットワーク（入力層を第０層と数えた場合）
def predict(network, x):
    W1, W2, W3 = network["W1"], network["W2"], network["W3"]
    b1, b2, b3 = network["b1"], network["b2"], network["b3"]

    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)

    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    
    a3 = np.dot(z2, W3) + b3
    y  = softmax(a3)
    
    return y

In [8]:
#################################################
#用意した関数群を用いて、試しに推論してみる
#################################################

x, t = get_data()
network = init_network()

accuracy_count = 0 #推論の正解カウント（accuracy 正確さ)

print("=== test of network ====")
print("data num = %d" % (len(x)))

#テスト画像の枚数だけ、ループする
for i in range(len(x)): 
    y = predict(network ,x[i])
    p = np.argmax(y) #最も確率の高い要素のインデックスを取得
    
    #print(y.shape)
    #print(y,p)
    
    if i % (len(x)/100) == 0:
        print("data %d/%d done. info=%d/%d" % (i, len(x), accuracy_count, len(x)))
        
    if p == t[i]: #得らたインデックスと正解を比較する。正解だったら、カウントアップ
        accuracy_count += 1
        
print("推論終了") #推論に結構サクッと終わる
print("ニューラルネットワークの正答率==%f" % (accuracy_count/len(x)) )

=== test of network ====
data num = 10000
data 0/10000 done. info=0/10000
data 100/10000 done. info=96/10000
data 200/10000 done. info=194/10000
data 300/10000 done. info=287/10000
data 400/10000 done. info=377/10000
data 500/10000 done. info=471/10000
data 600/10000 done. info=560/10000
data 700/10000 done. info=653/10000
data 800/10000 done. info=748/10000
data 900/10000 done. info=841/10000
data 1000/10000 done. info=932/10000
data 1100/10000 done. info=1024/10000
data 1200/10000 done. info=1114/10000
data 1300/10000 done. info=1201/10000
data 1400/10000 done. info=1293/10000
data 1500/10000 done. info=1383/10000
data 1600/10000 done. info=1473/10000
data 1700/10000 done. info=1564/10000
data 1800/10000 done. info=1651/10000
data 1900/10000 done. info=1743/10000
data 2000/10000 done. info=1831/10000
data 2100/10000 done. info=1921/10000
data 2200/10000 done. info=2009/10000
data 2300/10000 done. info=2105/10000
data 2400/10000 done. info=2197/10000
data 2500/10000 done. info=2289/10

In [9]:
#############################
#バッチを使った例
#############################
x, t = get_data()
network = init_network()

accuracy_count = 0 #推論の正解カウント（accuracy 正確さ)
batch_size = 1000

print("=== test of network ====")
print("data num = %d" % (len(x)))

#テスト画像の枚数だけ、ループする
for i in range(0, len(x), batch_size): 
    y = predict(network ,x[i:i+batch_size])
    p = np.argmax(y, axis=1) #最も確率の高い要素のインデックスを取得。ただし、複数行になるため、行毎(axis=1)にＭＡＸなインデックスを得る
            
    accuracy_count += np.sum( p == t[i:i+batch_size]) #得らたインデックスと正解を比較する。正解だったら、カウントアップ。バッチなので、numpyのブロードキャストを使う
        
print("推論終了") #推論に結構サクッと終わる
print("ニューラルネットワークの正答率==%f" % (accuracy_count/len(x)) )


=== test of network ====
data num = 10000
推論終了
ニューラルネットワークの正答率==0.935200
